In [35]:
import torch.nn.functional as F
import torch
from GridMLM_tokenizers import GuidedGridMLMTokenizer
from models import GuidedMLMH
from generate_utils import load_model
import pickle
import pandas as pd
import os

In [36]:
curricula = ['random', 'base2']
subfolders = ['unf_CA', 'disentangle/unf_CA']
ablations = ['all', 'kl', 'rec', 'con']

device_name = 'cuda:2'

tokenizer = GuidedGridMLMTokenizer(fixed_length=256)
model = load_model(
        curriculum_type=curricula[0],
        subfolder=subfolders[0],
        ablation=ablations[0],
        device_name=device_name,
        tokenizer=tokenizer
)
device = model.device

In [37]:
save_folder = '/media/maindisk/data/hooktheory_hr/guidance_experiment_data/'
files = os.listdir(save_folder)
print(files)

['unf_CA_random_all_test_test.pickle', 'unf_CA_base2_kl_jazz_jazz.pickle', 'unf_CA_random_rec_test_test.pickle', 'unf_CA_random_kl_test_jazz.pickle', 'unf_CA_random_con_test_test.pickle', 'unf_CA_base2_kl_test_jazz.pickle', 'unf_CA_base2_con_test_test.pickle', 'unf_CA_base2_kl_jazz_test.pickle', 'unf_CA_random_rec_test_jazz.pickle', 'unf_CA_base2_all_jazz_test.pickle', 'unf_CA_base2_all_test_jazz.pickle', 'unf_CA_random_kl_jazz_test.pickle', 'unf_CA_random_kl_jazz_jazz.pickle', 'unf_CA_random_rec_jazz_jazz.pickle', 'unf_CA_base2_con_jazz_test.pickle', 'unf_CA_base2_kl_test_test.pickle', 'unf_CA_base2_con_jazz_jazz.pickle', 'unf_CA_random_all_jazz_jazz.pickle', 'unf_CA_random_con_test_jazz.pickle', 'unf_CA_base2_con_test_jazz.pickle', 'unf_CA_base2_rec_jazz_test.pickle', 'unf_CA_base2_all_test_test.pickle', 'unf_CA_random_con_jazz_test.pickle', 'unf_CA_random_kl_test_test.pickle', 'unf_CA_random_all_test_jazz.pickle', 'unf_CA_base2_rec_jazz_jazz.pickle', 'unf_CA_base2_rec_test_test.pick

In [38]:
file = 'unf_CA_random_all_test_test.pickle'
with open(save_folder+file, 'rb') as f:
    print(save_folder+file)
    ds = pickle.load(f)

/media/maindisk/data/hooktheory_hr/guidance_experiment_data/unf_CA_random_all_test_test.pickle


In [39]:
# for file in files:
#     with open(save_folder+file, 'rb') as f:
#         print(save_folder+file)
#         ds = pickle.load(f)

In [47]:
# d = ds[0]
print(len(ds))
d = ds[0]
print(d.keys())
original_ids = d['input_encoded']['input_ids']
results = d['results']
r = results[3]
guiding_ids = r['guide_encoded']['input_ids']
gen_ids = r['generated']['input_ids']
print('original_ids:', original_ids)
print('guiding_ids:', guiding_ids)
print('gen_ids:', gen_ids)

z_original = model.get_z_from_harmony(torch.LongTensor([original_ids]).to(device)).detach().cpu()[0].tolist()
z_guiding = model.get_z_from_harmony(torch.LongTensor([guiding_ids]).to(device)).detach().cpu()[0].tolist()
z_gen = model.get_z_from_harmony(torch.LongTensor([gen_ids]).to(device)).detach().cpu()[0].tolist()

guide_z = F.cosine_similarity(torch.FloatTensor(z_original), torch.FloatTensor(z_gen), dim=-1)
input_z = F.cosine_similarity(torch.FloatTensor(z_guiding), torch.FloatTensor(z_gen), dim=-1)
input_guide = F.cosine_similarity(torch.FloatTensor(z_guiding), torch.FloatTensor(z_original), dim=-1)
print('input_z:', input_z)
print('guide_z:', guide_z)
print('input_guide:', input_guide)

100
dict_keys(['input_encoded', 'results'])
original_ids: [65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
guiding_ids: [4, 4, 4, 4, 4, 4, 4, 4, 268, 268, 268, 